# Encontrando los datos atípicos

In [15]:
import pandas as pd

df = pd.read_csv (r'datos-5a6.csv')
df

,Estacion,Ubicacion,Proyecto,Longitud,Latitud,Radiacion,Fecha,Hora,Horas,Minutos,Valores X
0,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,127.8,2018-03-13,17:04,17,4,1704
1,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,87.3,2018-03-13,17:09,17,9,1709
2,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,43.8,2018-03-13,17:14,17,14,1714
3,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,34.7,2018-03-13,17:19,17,19,1719
4,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,31.0,2018-03-13,17:24,17,24,1724
...,...,...,...,...,...,...,...,...,...,...,...
43986,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,29.4,2018-03-12,17:44,17,44,1744
43987,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,12.2,2018-03-12,17:49,17,49,1749
43988,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,4.2,2018-03-12,17:54,17,54,1754
43989,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,95.0,2018-03-12,17:04,17,4,1704


In [16]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%Y/%m/%d")
df.dtypes

Estacion             object
Ubicacion            object
Proyecto             object
Longitud            float64
Latitud             float64
Radiacion           float64
Fecha        datetime64[ns]
Hora                 object
Horas                 int64
Minutos               int64
Valores X             int64
dtype: object

In [17]:
df = df.sort_values(by=['Fecha'])
df

,Estacion,Ubicacion,Proyecto,Longitud,Latitud,Radiacion,Fecha,Hora,Horas,Minutos,Valores X
32779,Norte,Bucaramanga,CDMB,7.149110,-73.134533,32.00,2010-01-01,17:00,17,0,1700
31408,Centro,Bucaramanga,CDMB,7.119262,-73.127154,40.00,2010-01-01,17:00,17,0,1700
31409,Centro,Bucaramanga,CDMB,7.119262,-73.127154,42.00,2010-01-02,17:00,17,0,1700
32780,Norte,Bucaramanga,CDMB,7.149110,-73.134533,33.00,2010-01-02,17:00,17,0,1700
32781,Norte,Bucaramanga,CDMB,7.149110,-73.134533,39.00,2010-01-03,17:00,17,0,1700
...,...,...,...,...,...,...,...,...,...,...,...
29729,UPB - Piedecuesta,Piedecuesta,Wunderground,7.038574,-73.069649,82.11,2021-01-26,17:59:47,17,59,1759
29731,UPB - Piedecuesta,Piedecuesta,Wunderground,7.038574,-73.069649,122.08,2021-01-27,17:59:47,17,59,1759
29732,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,113.59,2021-01-27,17:39:03,17,39,1739
29733,UPB - Piedecuesta,Piedecuesta,Wunderground,7.038574,-73.069649,106.02,2021-01-28,17:59:47,17,59,1759


In [18]:
df.Estacion.unique()

array(['Norte', 'Centro', 'Florida', 'Ciudadela', 'Paralela Bosque',
       'UPB - Piedecuesta'], dtype=object)

Sacar df por cada estación ya ordenados por fecha

In [19]:
df_PB = df[(df.Estacion == 'Paralela Bosque')]
df_UPB = df[(df.Estacion == 'UPB - Piedecuesta')]
df_ciu = df[(df.Estacion == 'Ciudadela')]
df_cen = df[(df.Estacion == 'Centro')]
df_nor = df[(df.Estacion == 'Norte')]
df_flo = df[(df.Estacion == 'Florida')]

df_PB = df_PB.reset_index(drop=True)
df_UPB = df_UPB.reset_index(drop=True)
df_ciu = df_ciu.reset_index(drop=True)
df_cen = df_cen.reset_index(drop=True)
df_nor = df_nor.reset_index(drop=True)
df_flo = df_flo.reset_index(drop=True)

Creación de método de interpolación de Lagrange. Recibe el grado del polinomio, el vector de los valores de x, el vector de los valores de f(x) y la x a interpolar.

In [20]:
#lenVec = longitud del vector de datos,vecX= horas, vecFX= radiación, X= dato atipico a arreglar
def lagrange(lenVec,vecX,vecFX,x): 
    fx = 0
    for i in range(lenVec):
        multi = vecFX[i] #almacena el valor de f(xi) por los terminos de las multiplicaciones sucesivas
        for j in range(lenVec):
            if(j!=i):
                num = x - vecX[j]
                den = vecX[i] - vecX[j]
                multi = multi * (num/den)
        fx = fx + multi #fx hace las sumatorias de Li(x)f(xi)
    return fx

Creación de método de interpolación lineal.

In [21]:
def lineal(x1,x2,fx1,fx2,x):
    fx = fx1 + (((fx2-fx1)/(x2-x1))*(x-x1))
    return fx

Se crea una función que encuentra y arregla los datos atipicos mediante lagrange, de un df dado.

In [22]:
import statistics

In [23]:
def arreglo_datos_atipicos(df_x):

    fecha_temp = df_x.loc[0,"Fecha"]
    rango_dia = []
    rango_radiacion = []
    for i in range(len(df_x)): 
        fecha_actual = df_x.loc[i,"Fecha"]
        if(fecha_temp != fecha_actual):
            #Sacar datos atipicos de la lista de radiaciones rango_dia
            if(len(rango_dia)>=3): #si hay dos datos o menos no hay atipicos

                zip_iterator = zip(rango_dia, rango_radiacion)
                xFx_dict = dict(zip_iterator)

                xFx_dict = dict(sorted(xFx_dict.items(), key=lambda item: item[1]))

                rango_radiacion.sort()

                pq1 = (len(rango_radiacion)+1)/4
                ent1 = int(pq1)
                dec1 = pq1 - int(pq1)
                if(ent1+1 >= len(rango_radiacion)):
                    q1 = rango_radiacion[ent1]
                else:
                    q1 = rango_radiacion[ent1] + (dec1*(rango_radiacion[ent1+1]-rango_radiacion[ent1])) #check que ent1 no se pase de la longitud

                pq3 = (3*(len(rango_radiacion)+1))/4
                ent3 = int(pq3)
                dec3 = pq3 - int(pq3)

                if(ent3+1 >= len(rango_radiacion)):
                    q3 = rango_radiacion[len(rango_radiacion)-1]
                else:
                    q3 = rango_radiacion[ent3] + (dec3*(rango_radiacion[ent3+1]-rango_radiacion[ent3])) #check que ent3 no se pase de la longitud

                IQR = q3-q1

                l1 = q1 - (1.5*IQR)
                l3 = q3 + (1.5*IQR)


                
                
                xFx_dict = dict(sorted(xFx_dict.items(), key=lambda item: item[0])) #ordenar por valores de X
                xVect = list(xFx_dict.keys()) 
                fxVect = list(xFx_dict.values())

                '''
                print(fecha_actual)
                print(xVect)
                print(fxVect)
                '''
                horas_atipicas = []
                horas_ant = []
                horas_sig = []
                
                contPos = 0
                for hora in xFx_dict:
                    
                    if(xFx_dict[hora] < l1 or xFx_dict[hora] > l3): #comprobación para saber si es un dato atipico
                        if(not((hora == xVect[0]) or (hora == xVect[len(xVect)-1]))): #si NO es la primera ni la ultima posición
                        
                            ant = xVect[contPos-1]
                            sig = xVect[contPos+1]
                            horas_ant.append(ant)
                            horas_sig.append(sig)
                            
                            horas_atipicas.append(hora)
                            
                            xVect.remove(hora)
                            fxVect.remove(xFx_dict[hora])
                            
                            contPos -= 1
                    
                    contPos += 1 

                contPos2 = 0
                for dato in horas_atipicas:
                    '''
                    print(fecha_actual)
                    print(xVect)
                    print(fxVect)
                    '''
                    datofx_arreglado = lagrange(len(xVect),xVect,fxVect,dato) 
                    
                    desv_estan = statistics.stdev(fxVect)
                    
                    if(datofx_arreglado > (3*desv_estan) or datofx_arreglado < 0):
                        ant = horas_ant[contPos2]
                        sig = horas_sig[contPos2]
                        datofx_arreglado = lineal(ant,sig,xFx_dict[ant],xFx_dict[sig],dato) # x1,x2,fx1,fx2,x
                    '''
                    print(dato)
                    print(datofx_arreglado)
                    '''
                    flag1 = (df.Estacion==df_x.loc[i,"Estacion"])
                    flag2 = (df.Fecha==fecha_actual)
                    flag3 = (df["Valores X"]==dato)
                    df.loc[flag1 & flag2 & flag3, 'Radiacion'] = datofx_arreglado
                    df
                    #print()
                    contPos2 += 1
                    


            #vaciar lista y continuar con la siguiente fecha
            rango_dia = []
            rango_radiacion = []
            xFx_dict = {}
            fecha_temp = fecha_actual

        rango_radiacion.append(df_x.loc[i,"Radiacion"]) 
        rango_dia.append(df_x.loc[i,"Valores X"])


Utilización del metodo para arreglar los datos del dataframe de Paralela Bosque y de UPB. Los demás registros no se pueden arreglar dado que solo tienen un registro por día, por lo que no se puede saber si tienen o no datos atipicos.

In [24]:
arreglo_datos_atipicos(df_PB)

In [25]:
arreglo_datos_atipicos(df_UPB)

In [26]:
df.to_csv('datos_arreglados.csv', encoding='utf-8',index=False)

# Estimación temporal empírica

In [29]:
import time

dfTimes = {"n":"Time(ns)"}



for i in range(100,len(df_PB),100):
    start = time.perf_counter_ns()
    arreglo_datos_atipicos(df_PB.head(i))
    end = time.perf_counter_ns()
    difference = end-start
    dfTimes[i] = difference
    

dfTimes

{'n': 'Time(ns)',
 100: 3608900,
 200: 6442100,
 300: 8774900,
 400: 11103900,
 500: 14869100,
 600: 23898100,
 700: 36400300,
 800: 35043900,
 900: 24817200,
 1000: 31026000,
 1100: 34840200,
 1200: 39034900,
 1300: 41613200,
 1400: 45075200,
 1500: 48589600,
 1600: 51818700,
 1700: 55498700,
 1800: 63446700,
 1900: 64670000,
 2000: 75490600,
 2100: 78475500,
 2200: 81136400,
 2300: 84944700,
 2400: 90643500,
 2500: 91323500,
 2600: 100370700,
 2700: 110203400,
 2800: 111088000,
 2900: 115191200,
 3000: 118004600,
 3100: 118440700,
 3200: 117144800,
 3300: 120271300,
 3400: 126975900,
 3500: 134401900,
 3600: 134868900,
 3700: 135937100,
 3800: 138334900,
 3900: 137702200,
 4000: 146959200,
 4100: 155399500,
 4200: 145345300,
 4300: 150079900,
 4400: 159206400,
 4500: 162797400,
 4600: 166539700,
 4700: 173750400,
 4800: 175857400,
 4900: 187172500,
 5000: 188416300,
 5100: 205040200,
 5200: 218140200,
 5300: 236084600,
 5400: 236581400,
 5500: 225074500,
 5600: 226474200,
 5700: 2293

In [30]:
with open('tiempos3.csv', 'w', encoding='utf-8') as f:
    for key in dfTimes.keys():
        f.write("%s,%s\n"%(key,dfTimes[key]))
